After further investigation, it seems I need a different approach to qwen3-Embedding-0.6B. I need to separate "queries" with instructions from "documents". The structure is assymetrical unlike the regular cosine similarity which compares the same rows and columns to each other.

In [5]:
import pandas as pd

In [7]:
sg_df_clean = pd.read_csv("sg_df_clean.csv")

In [8]:
import torch
import gc
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load Model 
model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B", 
    device="cuda", 
    model_kwargs={"torch_dtype": torch.bfloat16}
)
model.max_seq_length = 512 

# Setup Data
raw_texts = sg_df_clean['about_the_game'].astype(str).tolist()

# The specific 'Instruction' for game mechanics
instruction = "Represent this game primarily by its core gameplay mechanics and functional loops, treating the setting and narrative theme as secondary features for similarity matching\nQuery: "

# Encode "Rows" (The Searchers)
print("Encoding Queries (Rows)...")
query_embeddings = model.encode(
    [instruction + t for t in raw_texts], # Prepend instruction to everything
    batch_size=64, 
    normalize_embeddings=True,
    convert_to_tensor=True
)

# 4 Encode "Columns" (The Targets)
print("Encoding Documents (Columns)...")
doc_embeddings = model.encode(
    raw_texts, # Raw text ONLY
    batch_size=64,
    normalize_embeddings=True,
    convert_to_tensor=True
)

#find similarity
print("Computing Matrix...")
sim_matrix_tensor = model.similarity(query_embeddings, doc_embeddings)
print("Done")

`torch_dtype` is deprecated! Use `dtype` instead!


Encoding Queries (Rows)...
Encoding Documents (Columns)...
Computing Matrix...


In [11]:
del model, query_embeddings, doc_embeddings #saving memory
gc.collect()
torch.cuda.empty_cache()

In [13]:
# move to CPU, and convert to NumPy
# We use .detach() to ensure the tensor is separated from any computational graph
sim_matrix_np = sim_matrix_tensor.to(torch.float32).cpu().detach().numpy()

# Convert the resulting NumPy array directly into  DataFrame
sim_matrix_df = pd.DataFrame(sim_matrix_np)

In [14]:
del sim_matrix_tensor, sim_matrix_np
gc.collect()
torch.cuda.empty_cache()

__Cosine Similarity__

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
tfidf_matrix = pd.read_pickle("TF-IDF_V1.pkl")

In [21]:
sim_matrix_tfidf = cosine_similarity(tfidf_matrix) #cosine similarity for tf-idf

In [22]:
del tfidf_matrix

In [23]:
import gc
gc.collect() 

2936

In [24]:
del sg_df_clean

In [29]:
sim_matrix_tfidf = sim_matrix_tfidf.astype(np.float32)

In [30]:
tfidf_w = 0.3
embeddings_w = 0.7

In [31]:
weighted_matrix_1 = tfidf_w * sim_matrix_tfidf

In [32]:
del sim_matrix_tfidf
gc.collect()

4110

In [37]:
weighted_matrix_2 =embeddings_w * sim_matrix_df

In [44]:
del sim_matrix_df

In [46]:
gc.collect()

5618

In [47]:
final_matrix = weighted_matrix_1 + weighted_matrix_2

In [49]:
del weighted_matrix_1 
del weighted_matrix_2

In [50]:
pd.to_pickle(final_matrix, "Full_cosine_matrix_Qwen-3-6B-V3.pkl")

I will skip testing this attempt individually and compare all models in the next document